In [31]:
# chr-pos - rs_id 字典
rs_dict = {}
with open("/data/mengxf/Project/KML250324_drSNP_iGT/work/250324/target/A175loci-hg19.freq-anno_with_ref_alt.tsv") as f:
    for line in f:
        lns = line.strip().split("\t")
        """
        {('chr1', '14155402'): ['rs7520386', 'G', 'A'],
        """
        rs_dict[(lns[0], lns[1])] = (lns[2:])
rs_dict

{('chr1', '14155402'): ['rs7520386', 'G', 'A'],
 ('chr1', '89388944'): ['rs7532151', 'A', 'C'],
 ('chr1', '107984098'): ['rs776284', 'A', 'G'],
 ('chr1', '120612006'): ['rs4021006', 'G', 'A'],
 ('chr1', '160786670'): ['rs560681', 'A', 'G'],
 ('chr1', '162737116'): ['rs2298258', 'C', 'G'],
 ('chr1', '233448413'): ['rs1294331', 'C', 'T'],
 ('chr10', '5721653'): ['rs2386626', 'C', 'A'],
 ('chr10', '17193346'): ['rs3780962', 'A', 'G'],
 ('chr10', '43595968'): ['rs1800858', 'A', 'G'],
 ('chr10', '43613843'): ['rs1800861', 'G', 'T'],
 ('chr10', '92792388'): ['rs7907658', 'C', 'A'],
 ('chr10', '104855656'): ['rs1926030', 'T', 'C'],
 ('chr10', '118506899'): ['rs740598', 'G', 'A'],
 ('chr11', '5709028'): ['rs1498553', 'C', 'T'],
 ('chr11', '64201639'): ['rs550840', 'C', 'T'],
 ('chr11', '115207176'): ['rs10488710', 'C', 'G'],
 ('chr12', '6945914'): ['rs2269355', 'C', 'G'],
 ('chr12', '130761696'): ['rs10773760', 'A', 'G'],
 ('chr13', '100038233'): ['rs1058083', 'A', 'G'],
 ('chr14', '62750546')

In [32]:
def get_alt_ao_af(alt, ao, chrom, pos, dp):
    rs_alt = rs_dict[(chrom, pos)][-1]
    # 处理多等位基因情况
    if ',' in alt:
        alt_list = alt.split(",")
        ao_list = ao.split(",")
        ao = ao_list[alt_list.index(rs_alt)] if rs_alt in alt_list else "0"
        alt = rs_alt
    # 处理其他情况
    elif alt == "." or alt != rs_alt:
        alt = rs_alt
        ao = "0"
    af = format(int(ao) / int(dp), ".6f")
    return alt, ao, af


vcf_dict = {}
with open("/data/mengxf/Project/KML250324_drSNP_iGT/work/250324/result/vcf/8434A1/8434A1.freebayes.vcf") as f:
    for line in f:
        if line.startswith("##"):
            continue
        elif line.startswith("#CHROM"):
            headers = line.strip().split("\t")
            hdr = {hie[1]: hie[0] for hie in enumerate(headers)}
        else:
            lns = line.strip().split("\t")
            chrom = lns[hdr["#CHROM"]]
            pos = lns[hdr["POS"]]
            ref = lns[hdr["REF"]]
            alt = lns[hdr["ALT"]]
            info = lns[hdr["INFO"]]
            info_dict = {i.split("=")[0]: i.split("=")[1] for i in info.split(";")}
            dp = info_dict["DP"]
            ao = info_dict["AO"] if "AO" in info_dict else "0"
            alt, ao, af = get_alt_ao_af(alt, ao, chrom, pos, dp)
            vcf_dict[(chrom, pos)] = [chrom, pos, ref, alt, dp, af]
vcf_dict

{('chr1', '14155402'): ['chr1', '14155402', 'G', 'A', '24669', '0.999554'],
 ('chr1', '89388944'): ['chr1', '89388944', 'A', 'C', '29149', '0.506021'],
 ('chr1', '107984098'): ['chr1', '107984098', 'A', 'G', '48727', '0.494757'],
 ('chr1', '120612006'): ['chr1', '120612006', 'G', 'A', '11629', '0.500989'],
 ('chr1', '160786670'): ['chr1', '160786670', 'A', 'G', '23165', '0.999309'],
 ('chr1', '162737116'): ['chr1', '162737116', 'C', 'G', '29115', '0.000000'],
 ('chr1', '233448413'): ['chr1', '233448413', 'C', 'T', '41670', '0.000408'],
 ('chr10', '5721653'): ['chr10', '5721653', 'C', 'A', '38189', '0.000000'],
 ('chr10', '17193346'): ['chr10', '17193346', 'A', 'G', '21786', '0.999679'],
 ('chr10', '43595968'): ['chr10', '43595968', 'A', 'G', '21825', '0.479221'],
 ('chr10', '43613843'): ['chr10', '43613843', 'G', 'T', '19354', '0.999690'],
 ('chr10', '92792388'): ['chr10', '92792388', 'C', 'A', '18201', '0.472227'],
 ('chr10', '104855656'): ['chr10', '104855656', 'T', 'C', '24547', '0.

In [33]:
with open("../result/frbys_snp_anno.tsv", "w") as f:
    f.write("#RS_ID\tCHROM\tPOS\tREF\tALT\tDP\tAF\n")
    for k in rs_dict:
        rs_id = rs_dict[k]
        f.write('\t'.join([rs_dict[k][0]] + vcf_dict[k]) + "\n")